In [1]:
import selenium 
import csv
import re
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options

import time

In [2]:
sample_df = pd.read_csv("../PubChem Scraper/pubchem_longer_wait_final.csv")
# sample_df = sample_df.sample(200)

cas_df = sample_df[sample_df["CAS"] != "No CAS info"]
missing_df = sample_df[sample_df["CAS"] == "No CAS info"] 
sample_df

,Original Molecule,Parsed Molecule,Link,Result Type,CAS,Deprecated CAS,Compound/Substance,Depositor Synonym List,Regular Synonym List,Synonym?
0,"1,2 PROPANEDIOL DIACETATE","1,2 PROPANEDIOL DIACETATE",https://pubchem.ncbi.nlm.nih.gov/compound/12198,RELEVANT,623-84-7,"134236-23-0, 1432741-27-9",COMPOUND,YES,YES,YES
1,1 HEXADECANOL,1 HEXADECANOL,https://pubchem.ncbi.nlm.nih.gov/compound/2682,RELEVANT,"36653-82-4, 36311-34-9, 124-29-8","168679-13-8, 1173838-87-3, 124-29-8, 55069-45-...",COMPOUND,YES,YES,YES
2,1 OCTADECANOL,1 OCTADECANOL,https://pubchem.ncbi.nlm.nih.gov/compound/8221,RELEVANT,"112-92-5, 26762-44-7","193766-48-2, 8014-37-7, 8032-19-7, 8032-21-1, ...",COMPOUND,YES,YES,YES
3,"2,3-(2 IODOPROPYLIDENEDIOXY)PROPANOL","2,3-(2 IODOPROPYLIDENEDIOXY)PROPANOL",No Link,Not Found,No CAS info,No CAS info,NEITHER,MISSING,MISSING,NO
4,2 OXOGLUTARIC ACID,2 OXOGLUTARIC ACID,https://pubchem.ncbi.nlm.nih.gov/compound/51,FEATURED,"328-50-7, 34410-46-3, 17091-15-5",27175-99-1,COMPOUND,YES,YES,YES
...,...,...,...,...,...,...,...,...,...,...
2900,ZIZYPHUS JUJUBA,ZIZYPHUS JUJUBA,https://pubchem.ncbi.nlm.nih.gov/substance/482...,RELEVANT,No CAS info,No CAS info,SUBSTANCE,NO,MISSING,NO
2901,ZOLEDRONIC ACID,ZOLEDRONIC ACID,https://pubchem.ncbi.nlm.nih.gov/compound/68740,FEATURED,118072-93-8,No CAS info,COMPOUND,YES,YES,YES
2902,ZOLMITRIPTAN,ZOLMITRIPTAN,https://pubchem.ncbi.nlm.nih.gov/compound/60857,FEATURED,139264-17-8,No CAS info,COMPOUND,YES,YES,YES
2903,ZOLPIDEM,ZOLPIDEM,https://pubchem.ncbi.nlm.nih.gov/compound/5732,FEATURED,82626-48-0,No CAS info,COMPOUND,YES,YES,YES


In [3]:
cas_df = cas_df[['Parsed Molecule', 'CAS']]
missing_df = missing_df[['Parsed Molecule', 'CAS']]

In [4]:
cas_df

,Parsed Molecule,CAS
0,"1,2 PROPANEDIOL DIACETATE",623-84-7
1,1 HEXADECANOL,"36653-82-4, 36311-34-9, 124-29-8"
2,1 OCTADECANOL,"112-92-5, 26762-44-7"
4,2 OXOGLUTARIC ACID,"328-50-7, 34410-46-3, 17091-15-5"
6,2 PROPANOL,"67-63-0, 3958-66-5"
...,...,...
2899,ZIRCONIUM CYCLOSILICATE,17141-74-1
2901,ZOLEDRONIC ACID,118072-93-8
2902,ZOLMITRIPTAN,139264-17-8
2903,ZOLPIDEM,82626-48-0


In [5]:
missing_df

,Parsed Molecule,CAS
3,"2,3-(2 IODOPROPYLIDENEDIOXY)PROPANOL",No CAS info
5,2 PHENOXYETHANOL,No CAS info
8,7 KETO DEHYDRANDROSTERONE,No CAS info
18,ABRUS PRECATORIUS,No CAS info
19,ABSORBABLE GELATIN/COLLAGEN SPONGE,No CAS info
...,...,...
2883,YELLOW BEESWAX,No CAS info
2885,YUCCA,No CAS info
2891,ZEA MAIS,No CAS info
2896,ZINGIBER OFFICINALE,No CAS info


In [6]:
def get_first_cas(cas_string):
    # Replace all semicolons with commas
    modified_string = re.sub(r';', ',', cas_string)
    
    if modified_string == "No CAS info":
        return None
    
    return modified_string.split(',')[0].strip()

# Apply this function to the entire CAS column
cas_df['CAS'] = cas_df['CAS'].apply(get_first_cas)

molecules = list(zip(cas_df['Parsed Molecule'], cas_df['CAS']))
molecules

# missing_df['CAS'] = missing_df['CAS'].apply(get_first_cas)
# missing_arr = missing_df['CAS'].tolist()

[('1,2 PROPANEDIOL DIACETATE', '623-84-7'),
 ('1 HEXADECANOL', '36653-82-4'),
 ('1 OCTADECANOL', '112-92-5'),
 ('2 OXOGLUTARIC ACID', '328-50-7'),
 ('2 PROPANOL', '67-63-0'),
 ('4 AMINOBUTYRIC ACID', '56-12-2'),
 ('7 OXO-DEHYDROEPIANDROSTERONE', '566-19-8'),
 ('8 QUINOLINOL', '148-24-3'),
 ('ABACAVIR', '136470-78-5'),
 ('ABALOPARATIDE', '247062-33-5'),
 ('ABARELIX', '183552-38-7'),
 ('ABATACEPT', '332348-12-6'),
 ('ABCIXIMAB', '143653-53-6'),
 ('ABEMACICLIB', '1231929-97-7'),
 ('ABIRATERONE ACETATE', '154229-18-2'),
 ('ACALABRUTINIB', '1420477-60-6'),
 ('ACAMPROSATE', '77337-76-9'),
 ('ACARBOSE', '56180-94-0'),
 ('ACEBUTOLOL', '37517-30-9'),
 ('ACEMANNAN', '110042-95-0'),
 ('ACEPROMAZINE', '61-00-7'),
 ('ACESULFAME', '33665-90-6'),
 ('ACETAZOLAMIDE', '59-66-5'),
 ('ACETIC ACID', '64-19-7'),
 ('ACETOHEXAMIDE', '968-81-0'),
 ('ACETOHYDROXAMIC ACID', '546-88-3'),
 ('ACETONE', '67-64-1'),
 ('ACETYL-L-CARNITINE', '3040-38-8'),
 ('ACETYLCHOLINE', '51-84-3'),
 ('ACETYLCYSTEINE', '616-91-1'),


In [7]:
import urllib

def setup_webdriver():
    '''Initializes a headless selenium webdriver'''
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def cas_url_inxight(molecule):
    cas = molecule[1]
    cas_formatted = cas.replace('-', '%20')
    return f'https://drugs.ncats.io/substances?q=(root_codes_CAS:\"{cas_formatted}\")'

def name_url_inxight(molecule):
    '''Generates an Inxight URL for a given CAS'''
    name = molecule[0]
    cas = molecule[1]
    cas_formatted = cas.replace('-', '%20')
    name_encoded = urllib.parse.quote(f'^{name}')
    return f'https://drugs.ncats.io/substances?q=(root_codes_CAS:\"{cas_formatted}\")%20AND%20(root_names_name:\"{name_encoded}\")'

def name_exact_inxight(molecule):
    '''Generates an Inxight URL for a given CAS'''
    name = molecule[0]
    cas = molecule[1]
    cas_formatted = cas.replace('-', '%20')
    name_encoded = urllib.parse.quote(f'^{name}')
    return f'https://drugs.ncats.io/substances?q=(root_codes_CAS:\"{cas_formatted}\")%20AND%20(root_names_name:\"{name_encoded}$\")'

In [8]:
def get_inxight_url(molecule, driver):
    '''
    Parameters
    ----------
    A molecule's CAS and an initialised webdriver.

    Returns
    -------
    The top Inxight search result for a given CAS number
    '''
    
    url = "N/A"  # Default in case of failure
    query = cas_url_inxight(molecule)
    query_type = "ONLY CAS"
    driver.get(query)
    
    try:
        WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'span#record-count:nth-child(2)')))
        elements = driver.find_elements(By.CSS_SELECTOR, 'span#record-count:nth-child(2)')
        if elements:
            count = int(elements[0].text)
            print(count)
        else:
            print("Element not found.")
            
    except TimeoutException:
        print("Element not found within specified time.")
        count = 0

    if count > 1:
        query = name_exact_inxight(molecule)
        query_type = "EXACT NAME"
        driver.get(query)
    else:
        pass
        
    try:
        WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[id="card-title"]')))
        elements = driver.find_elements(By.CSS_SELECTOR, 'a[id="card-title"]')
        if elements:
            element = elements[0]
            url = element.get_attribute('href')

    except (NoSuchElementException, TimeoutException):
        query = name_url_inxight(molecule)
        query_type = "APPROXIMATE NAME"
        driver.get(query)
        
        try:
            WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[id="card-title"]')))
            elements = driver.find_elements(By.CSS_SELECTOR, 'a[id="card-title"]')
            if elements:
                element = elements[0]
                url = element.get_attribute('href')

        except (NoSuchElementException, TimeoutException):
            query = cas_url_inxight(molecule)
            query_type = "ONLY CAS"
            driver.get(query)

            try:
                WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[id="card-title"]')))
                elements = driver.find_elements(By.CSS_SELECTOR, 'a[id="card-title"]')
                if elements:
                    element = elements[0]
                    url = element.get_attribute('href')
            
            except (NoSuchElementException, TimeoutException):
                pass
            
    return molecule, url, count, query, query_type

def get_best_urls(mol_arr):
    '''
    Parameters
    ----------
    An array of CAS values

    Returns
    -------
    An array of associated top matches
    '''
    
    best_matches = []
    driver = setup_webdriver()  
    try:
        for molecule in mol_arr:
            molecule, url, count, query, query_type = get_inxight_url(molecule, driver)
            best_matches.append((molecule, url, count, query, query_type))
    finally:
        driver.quit()  

    return best_matches

In [9]:
import requests

def get_additional_data(id):
    r = requests.get(f'https://drugs.ncats.io/api/v1/substances({id})/@additional')
    if 200 == r.status_code:
        return r.json()
    return None

In [10]:
def extract_conditions_and_phases(data):
    conditions_list = []
    highest_approval_list = []

    if data is None:
        return conditions_list, highest_approval_list

    for entry in data:
        if entry['name'] == 'Conditions' and 'value' in entry:
            condition_info = entry['value']
            
            # Extract the condition name
            label = condition_info.get('label')
            if label:
                conditions_list.append(label)
            
            # Extract the highest phase of approval
            highest_phase = condition_info.get('highestPhase')
            if highest_phase:
                highest_approval_list.append(highest_phase)

    return conditions_list, highest_approval_list

In [11]:
# def extract_event_details(data):
#     event_details = {}
    
#     for item in data:
#         if 'value' in item and isinstance(item['value'], dict):  # Ensure 'value' is a dictionary
#             details = item['value']
#             if 'status' in details and 'sourceID' in details:
#                 if item['name'] == 'Highest Development Event' or item['name'] == 'Earliest Approved Event':
#                     # Gather additional details
#                     source_id = details.get('sourceID', 'No Source ID')
#                     source_url = details.get('sourceURL', 'No Source URL')
                    
#                     detail_info = {
#                         'Status and Year': f"{details['status']} {details.get('year', '')}",
#                         'Source ID': source_id,
#                         'Source URL': source_url
#                     }
                    
#                     # Use the 'name' of the event as the key in the dictionary
#                     event_details[item['name']] = detail_info
    
#     return event_details

def extract_event_details(data):
    event_details = {}
    if data is None:  # Check if data is None before iterating
        return event_details

    for item in data:
        if 'value' in item and isinstance(item['value'], dict):
            details = item['value']
            if 'status' in details and 'sourceID' in details:
                if item['name'] in ['Highest Development Event', 'Earliest Approved Event']:
                    detail_info = {
                        'Status and Year': f"{details['status']} {details.get('year', '')}",
                        'Source ID': details.get('sourceID', 'No Source ID'),
                        'Source URL': details.get('sourceURL', 'No Source URL')
                    }
                    event_details[item['name']] = detail_info
    return event_details

In [12]:
links = get_best_urls(molecules)
links

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Element not found within specified time.
1
1
1
1
7
Element not found within specified time.
1
1
1
1
1
1
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
Element not found within specified time.
1
1
1
1
1
Element not found within specified time.
Element not found within specified time.
1
1
1
1
1
1
1
1
1
1
1
Element not found within specified time.
1
1
1
1
1
1
1
1
1
1
3
1
Element not found within specified time.
Element not found within specified time.
13
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
Element not found within specified time.
1
2
1
1
1
1
1
Element not found within specified time.
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
4
1
2
1
1
1


[(('1,2 PROPANEDIOL DIACETATE', '623-84-7'),
  'https://drugs.ncats.io/drug/5Z492UNF9O',
  1,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"623%2084%207")',
  'ONLY CAS'),
 (('1 HEXADECANOL', '36653-82-4'),
  'https://drugs.ncats.io/drug/936JST6JCN',
  1,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"36653%2082%204")',
  'ONLY CAS'),
 (('1 OCTADECANOL', '112-92-5'),
  'https://drugs.ncats.io/drug/2KR89I4H1Y',
  1,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"112%2092%205")',
  'ONLY CAS'),
 (('2 OXOGLUTARIC ACID', '328-50-7'),
  'https://drugs.ncats.io/drug/8ID597Z82X',
  1,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"328%2050%207")',
  'ONLY CAS'),
 (('2 PROPANOL', '67-63-0'),
  'https://drugs.ncats.io/drug/ND2M416302',
  1,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"67%2063%200")',
  'ONLY CAS'),
 (('4 AMINOBUTYRIC ACID', '56-12-2'),
  'https://drugs.ncats.io/drug/2ACZ6IPC6I',
  1,
  'https://drugs.ncats.io/substances?q=(root_co

In [13]:
# new_links_df = pd.DataFrame(links, columns=['molecule', 'query_url', 'results', 'best_match_url', ''])
# new_links_df.head(2)
# links = new_links_df['best_match_url'].to_list()
# links
# identifiers = [url.split('/')[-1] for _, url in links]
# identifiers

data = []
for link in links:
    molecule_name, cas_number = link[0]
    drug_url, value, substance_url, query_type = link[1:]
    data.append([molecule_name, cas_number, drug_url, value, substance_url, query_type])
    
new_links_df = pd.DataFrame(data, columns=['molecule', 'cas', 'best_match_url', 'results', 'query_url', 'query_type'])
new_links_df.head(2)

,molecule,cas,best_match_url,results,query_url,query_type
0,"1,2 PROPANEDIOL DIACETATE",623-84-7,https://drugs.ncats.io/drug/5Z492UNF9O,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
1,1 HEXADECANOL,36653-82-4,https://drugs.ncats.io/drug/936JST6JCN,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS


In [14]:
identifier_arr = new_links_df['best_match_url'].to_list()
identifier_arr
identifiers = [url.split('/')[-1] for url in identifier_arr]

In [15]:
identifiers = [identifier if identifier != 'A' else 'MISSING' for identifier in identifiers]

In [16]:
len(identifiers)

2340

In [17]:
def extract_events(identifiers):
    data = []
    events = []

    for identifier in identifiers:
        data = get_additional_data(identifier)
        print(extract_event_details(data))
        events.append(extract_event_details(data))

    return events
        
events = extract_events(identifiers)

# def extract_events(identifiers):
#     events = []
#     for identifier in identifiers:
#         data = get_additional_data(identifier)
#         if data is not None:  # Check if data is None
#             event_details = extract_event_details(data)
#             events.append(event_details)
#         else:
#             print(f"No data available for identifier {identifier}")  # Or handle it differently
#     return events

{'Highest Development Event': {'Status and Year': 'Possibly Marketed Outside US 1996', 'Source ID': 'ANDA040166', 'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c0ec47b7-c13a-4fa0-91fe-d7a03c70aec9'}, 'Earliest Approved Event': {'Status and Year': 'Possibly Marketed Outside US 1996', 'Source ID': 'ANDA040166', 'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c0ec47b7-c13a-4fa0-91fe-d7a03c70aec9'}}
{'Highest Development Event': {'Status and Year': 'US Previously Marketed 1990', 'Source ID': 'EXOSURF NEONATAL by GLAXOSMITHKLINE', 'Source URL': 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=020044'}, 'Earliest Approved Event': {'Status and Year': 'Possibly Marketed Outside US 1984', 'Source ID': 'NU-DERM SUNFADER  Skin Lightener with Sunscreen (SPF 15) PABA FREE by OMP, INC.', 'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c67b0629-5529-412d-99ee-186df4ecad5a'}}
{'Highest Dev

In [18]:
def extract_conditions(identifiers):
    conditions = []

    for identifier in identifiers:
        data = get_additional_data(identifier)
        conditions.append(extract_conditions_and_phases(data))

    return conditions
        
conditions_arr = extract_conditions(identifiers)

In [19]:
events

[{'Highest Development Event': {'Status and Year': 'Possibly Marketed Outside US 1996',
   'Source ID': 'ANDA040166',
   'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c0ec47b7-c13a-4fa0-91fe-d7a03c70aec9'},
  'Earliest Approved Event': {'Status and Year': 'Possibly Marketed Outside US 1996',
   'Source ID': 'ANDA040166',
   'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c0ec47b7-c13a-4fa0-91fe-d7a03c70aec9'}},
 {'Highest Development Event': {'Status and Year': 'US Previously Marketed 1990',
   'Source ID': 'EXOSURF NEONATAL by GLAXOSMITHKLINE',
   'Source URL': 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=020044'},
  'Earliest Approved Event': {'Status and Year': 'Possibly Marketed Outside US 1984',
   'Source ID': 'NU-DERM SUNFADER  Skin Lightener with Sunscreen (SPF 15) PABA FREE by OMP, INC.',
   'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c67b0629-5529-412d-99e

In [20]:
conditions_arr

[(['Bacillus cereus infection'], ['Basic research']),
 (['Dry, itchy skin', 'Respiratory distress syndrome, neonatal'],
  ['Approved', 'Approved']),
 (['SUNBURN'], ['Approved']),
 (['Osteoporosis',
   'Gastrointestinal microbiota imbalance',
   'Short bowel syndrome',
   'Burn',
   'Kidney failure'],
  ['Not Provided', 'Not Provided', 'Not Provided', 'Approved', 'Approved']),
 ([], []),
 (['Type 1 diabetes mellitus',
   'Stress',
   'Vascular diseases of the brain',
   'Atherosclerosis of cerebral arteries',
   'Chronic cerebrovascular insufficiency',
   'Partial Seizures'],
  ['Phase II', 'Phase I', 'Approved', 'Approved', 'Approved', 'Not Provided']),
 ([], []),
 (['Unknown'], []),
 (['HIV-1 infection'], ['Approved']),
 (['Osteoporosis'], ['Approved']),
 (['Advanced prostate cancer'], ['Approved']),
 ([], []),
 ([], []),
 (['Breast cancer',
   'Melanoma',
   'Glioblastoma',
   'Non-small cell lung cancer',
   'HER2-negative advanced breast cancer'],
  ['Phase III', 'Phase II', 'Phase

In [21]:
data = []
for event in events:
    record = {}
    for key, value in event.items():
        for sub_key, sub_value in value.items():
            record[f"{key} {sub_key}"] = sub_value
    data.append(record)

df = pd.DataFrame(data)

In [22]:
df

,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL
0,Possibly Marketed Outside US 1996,ANDA040166,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Possibly Marketed Outside US 1996,ANDA040166,https://dailymed.nlm.nih.gov/dailymed/drugInfo...
1,US Previously Marketed 1990,EXOSURF NEONATAL by GLAXOSMITHKLINE,https://www.accessdata.fda.gov/scripts/cder/da...,Possibly Marketed Outside US 1984,NU-DERM SUNFADER Skin Lightener with Sunscree...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...
2,US Previously Marketed,21 CFR 310.545(a)(18)(i)(B) skin protectant:w/...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,Possibly Marketed Outside US 1970,Ala Quin by Crown Laboratories,https://dailymed.nlm.nih.gov/dailymed/drugInfo...
3,Possibly Marketed Outside US,ALPHA-KETOGLUTARATE,http://naturaldatabase.therapeuticresearch.com...,NaN,NaN,NaN
4,US Approved OTC,21 CFR 344.12 otic:ear drying aid isopropyl al...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,US Previously Marketed 1921,Rubbing Alcohol,https://books.google.com/books?id=1n4gAQAAMAAJ...
...,...,...,...,...,...,...
2335,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...
2336,US Approved Rx 2013,ANDA091363,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 2001,ZOMETA by NOVARTIS,https://www.accessdata.fda.gov/scripts/cder/da...
2337,US Approved Rx 2016,ANDA205074,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1997,ZOMIG by IPR,https://www.accessdata.fda.gov/scripts/cder/da...
2338,US Approved Rx 2016,ANDA201509,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1992,NDA019908,https://www.accessdata.fda.gov/scripts/cder/da...


In [23]:
rows = [{'conditions': conditions, 'phases': phases} for conditions, phases in conditions_arr]

# Create DataFrame
conditions_df = pd.DataFrame(rows)
conditions_df['conditions'] = conditions_df['conditions'].apply(lambda x: '; '.join(x))
conditions_df['phases'] = conditions_df['phases'].apply(lambda x: '; '.join(x))
conditions_df

,conditions,phases
0,Bacillus cereus infection,Basic research
1,"Dry, itchy skin; Respiratory distress syndrome...",Approved; Approved
2,SUNBURN,Approved
3,Osteoporosis; Gastrointestinal microbiota imba...,Not Provided; Not Provided; Not Provided; Appr...
4,,
...,...,...
2335,,
2336,Postmenopausal osteoporosis; Hypercalcemia; Pr...,Approved; Approved; Approved; Approved; Approved
2337,Migraine with aura; Migraine without aura,Approved; Approved
2338,Insomnia,Approved


In [24]:
merged = pd.concat([df, conditions_df], axis=1)
merged

,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,Possibly Marketed Outside US 1996,ANDA040166,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Possibly Marketed Outside US 1996,ANDA040166,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Bacillus cereus infection,Basic research
1,US Previously Marketed 1990,EXOSURF NEONATAL by GLAXOSMITHKLINE,https://www.accessdata.fda.gov/scripts/cder/da...,Possibly Marketed Outside US 1984,NU-DERM SUNFADER Skin Lightener with Sunscree...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,"Dry, itchy skin; Respiratory distress syndrome...",Approved; Approved
2,US Previously Marketed,21 CFR 310.545(a)(18)(i)(B) skin protectant:w/...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,Possibly Marketed Outside US 1970,Ala Quin by Crown Laboratories,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,SUNBURN,Approved
3,Possibly Marketed Outside US,ALPHA-KETOGLUTARATE,http://naturaldatabase.therapeuticresearch.com...,NaN,NaN,NaN,Osteoporosis; Gastrointestinal microbiota imba...,Not Provided; Not Provided; Not Provided; Appr...
4,US Approved OTC,21 CFR 344.12 otic:ear drying aid isopropyl al...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,US Previously Marketed 1921,Rubbing Alcohol,https://books.google.com/books?id=1n4gAQAAMAAJ...,,
...,...,...,...,...,...,...,...,...
2335,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...,,
2336,US Approved Rx 2013,ANDA091363,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 2001,ZOMETA by NOVARTIS,https://www.accessdata.fda.gov/scripts/cder/da...,Postmenopausal osteoporosis; Hypercalcemia; Pr...,Approved; Approved; Approved; Approved; Approved
2337,US Approved Rx 2016,ANDA205074,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1997,ZOMIG by IPR,https://www.accessdata.fda.gov/scripts/cder/da...,Migraine with aura; Migraine without aura,Approved; Approved
2338,US Approved Rx 2016,ANDA201509,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1992,NDA019908,https://www.accessdata.fda.gov/scripts/cder/da...,Insomnia,Approved


In [25]:
new_links_df

,molecule,cas,best_match_url,results,query_url,query_type
0,"1,2 PROPANEDIOL DIACETATE",623-84-7,https://drugs.ncats.io/drug/5Z492UNF9O,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
1,1 HEXADECANOL,36653-82-4,https://drugs.ncats.io/drug/936JST6JCN,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
2,1 OCTADECANOL,112-92-5,https://drugs.ncats.io/drug/2KR89I4H1Y,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
3,2 OXOGLUTARIC ACID,328-50-7,https://drugs.ncats.io/drug/8ID597Z82X,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
4,2 PROPANOL,67-63-0,https://drugs.ncats.io/drug/ND2M416302,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
...,...,...,...,...,...,...
2335,ZIRCONIUM CYCLOSILICATE,17141-74-1,https://drugs.ncats.io/drug/D652ZWF066,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
2336,ZOLEDRONIC ACID,118072-93-8,https://drugs.ncats.io/drug/70HZ18PH24,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
2337,ZOLMITRIPTAN,139264-17-8,https://drugs.ncats.io/drug/2FS66TH3YW,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
2338,ZOLPIDEM,82626-48-0,https://drugs.ncats.io/drug/7K383OQI23,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS


In [26]:
result = pd.concat([new_links_df, merged], axis=1)
result

,molecule,cas,best_match_url,results,query_url,query_type,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,"1,2 PROPANEDIOL DIACETATE",623-84-7,https://drugs.ncats.io/drug/5Z492UNF9O,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,Possibly Marketed Outside US 1996,ANDA040166,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Possibly Marketed Outside US 1996,ANDA040166,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Bacillus cereus infection,Basic research
1,1 HEXADECANOL,36653-82-4,https://drugs.ncats.io/drug/936JST6JCN,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,US Previously Marketed 1990,EXOSURF NEONATAL by GLAXOSMITHKLINE,https://www.accessdata.fda.gov/scripts/cder/da...,Possibly Marketed Outside US 1984,NU-DERM SUNFADER Skin Lightener with Sunscree...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,"Dry, itchy skin; Respiratory distress syndrome...",Approved; Approved
2,1 OCTADECANOL,112-92-5,https://drugs.ncats.io/drug/2KR89I4H1Y,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,US Previously Marketed,21 CFR 310.545(a)(18)(i)(B) skin protectant:w/...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,Possibly Marketed Outside US 1970,Ala Quin by Crown Laboratories,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,SUNBURN,Approved
3,2 OXOGLUTARIC ACID,328-50-7,https://drugs.ncats.io/drug/8ID597Z82X,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,Possibly Marketed Outside US,ALPHA-KETOGLUTARATE,http://naturaldatabase.therapeuticresearch.com...,NaN,NaN,NaN,Osteoporosis; Gastrointestinal microbiota imba...,Not Provided; Not Provided; Not Provided; Appr...
4,2 PROPANOL,67-63-0,https://drugs.ncats.io/drug/ND2M416302,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,US Approved OTC,21 CFR 344.12 otic:ear drying aid isopropyl al...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,US Previously Marketed 1921,Rubbing Alcohol,https://books.google.com/books?id=1n4gAQAAMAAJ...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,ZIRCONIUM CYCLOSILICATE,17141-74-1,https://drugs.ncats.io/drug/D652ZWF066,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...,,
2336,ZOLEDRONIC ACID,118072-93-8,https://drugs.ncats.io/drug/70HZ18PH24,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,US Approved Rx 2013,ANDA091363,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 2001,ZOMETA by NOVARTIS,https://www.accessdata.fda.gov/scripts/cder/da...,Postmenopausal osteoporosis; Hypercalcemia; Pr...,Approved; Approved; Approved; Approved; Approved
2337,ZOLMITRIPTAN,139264-17-8,https://drugs.ncats.io/drug/2FS66TH3YW,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,US Approved Rx 2016,ANDA205074,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1997,ZOMIG by IPR,https://www.accessdata.fda.gov/scripts/cder/da...,Migraine with aura; Migraine without aura,Approved; Approved
2338,ZOLPIDEM,82626-48-0,https://drugs.ncats.io/drug/7K383OQI23,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,US Approved Rx 2016,ANDA201509,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1992,NDA019908,https://www.accessdata.fda.gov/scripts/cder/da...,Insomnia,Approved


In [28]:
sample_output = result.to_csv("inxight_corrected_cas_real.csv", index = False)